<a href="https://colab.research.google.com/github/NourHawila/PHS-597---Deep-Learning/blob/master/CNN_classify_beans2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import TensorFlow and TensorFlow Datasets
import tensorflow_datasets as tfds
import tensorflow as tf

import os

In [2]:
# Getting the data
datasets, info = tfds.load('beans', with_info=True, as_supervised=True)

beans_train, beans_test = datasets['train'], datasets['test']

Shuffling and writing examples to /root/tensorflow_datasets/beans/0.1.0.incompleteN2ATOM/beans-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/beans/0.1.0.incompleteN2ATOM/beans-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/beans/0.1.0.incompleteN2ATOM/beans-test.tfrecord


Dataset beans downloaded and prepared to /root/tensorflow_datasets/beans/0.1.0. Subsequent calls will reuse this data.


In [3]:
# Defining some hyperparameters
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000
BATCH_SIZE = 32

In [4]:
# Normalizing the pixel values
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

In [6]:
# Batching the dataset and keeping a memory buffer for better performance
train_dataset = beans_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = beans_test.map(scale).batch(BATCH_SIZE)

In [7]:
# Building our model
model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(500, 500, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3)
  ])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [8]:
# Define the checkpoint directory to store the checkpoints
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [9]:
# Function for decaying the learning rate.

def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [10]:
# Defining the callbacks
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay)
]

In [11]:
# Training the model
import time

start = time.time()
model.fit(train_dataset, epochs=5, callbacks=callbacks)
end = time.time()
print("Time elapsed: {}".format(end-start))

Epoch 1/5
33/33 [==============================] - 42s 186ms/step - loss: 2.3181 - accuracy: 0.4538
Epoch 2/5
33/33 [==============================] - 5s 141ms/step - loss: 0.7213 - accuracy: 0.6932
Epoch 3/5
33/33 [==============================] - 5s 142ms/step - loss: 0.5110 - accuracy: 0.7964
Epoch 4/5
33/33 [==============================] - 5s 143ms/step - loss: 0.3384 - accuracy: 0.8686
Epoch 5/5
33/33 [==============================] - 5s 143ms/step - loss: 0.2825 - accuracy: 0.8939
Time elapsed: 62.84194493293762
